#### *Continuation of Cat 1*
# MAPPING EDUCATIONAL EQUITY: A DATA-DRIVEN ANALYSIS OF INFRASTRUCTURE AND RESOURCE DISTRIBUTION IN KENYA'S PRIMARY SCHOOLS

## Author: **MOBISA KWAMBOKA 222612**
#### Strathmore University 
#### DSA 8102: DATA MINING, STORAGE AND RETRIEVAL
#### DR. KENNEDY SENAGI
#### CAT 2
### **Date**: 3/8/2025

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px

data = pd.read_csv("C:/Users/Bina/Desktop/Machine-Learning/EDA/data_mining assignment/kenya_primary_schools.csv")
print(data.columns)
print(data.shape)

In [ ]:
# the columns X and Y are longitude and latitude respectively.
#  I will drop the ones at the end and rename the X and Y
# Drop Latitude and Longitude columns
data.drop(columns=['Latitude', 'Longitude'], inplace=True)
print(data.columns)

# aside from renaming X and Y, I will rename the other columns for constitency, clarity and descriptive, rmeove typos
data.rename(columns={
    'X': 'Longitude',
    'Y': 'Latitude',
    'FID': 'Record_ID',
    'Name_of_Sc': 'School_Name',
    'Level_': 'School_Level',
    'Status': 'School_Status',
    'SchSponsor': 'School_Sponsor',
    'Type1': 'Student_Gender_Type',
    'Type2': 'School_Day_Boarding_Type',
    'Type3': 'Student_Category',
    'PupilTeach': 'Pupil_Teacher_Ratio',
    'ClassrmRat': 'Pupil_Classroom_Ratio',
    'ToiletRati': 'Pupil_Toilet_Ratio',
    'No_Classrm': 'Num_Classrooms',
    'BoysToilet': 'Num_Boys_Toilets',
    'GirlsToilet': 'Num_Girls_Toilets',
    'TeachersTo': 'Num_Teachers_Toilets',
    'TotalToile': 'Num_Toilets_Total',
    'TotalBoys': 'Num_Boys',
    'TotalGirls': 'Num_Girls',
    'TotalEnrol': 'Total_Enrollment',
    'GO_KTSC_M': 'GOK_TSC_Teachers_Male',
    'GOK_TSC_F': 'GOK_TSC_Teachers_Female',
    'authorityM': 'Authority_Teachers_Male',
    'authorityF': 'Authority_Teachers_Female',
    'PTA_BOG_M': 'PTA_BOG_Teachers_Male',
    'PTA_BOG_F': 'PTA_BOG_Teachers_Female',
    'OthersM': 'Other_Teachers_Male',
    'OthersF': 'Other_Teachers_Female',
    'NoTeaching': 'Num_Teaching_Staff',
    'NonTeachin': 'Num_Non_Teaching_Staff',
    'Province': 'Province',
    'District': 'District',
    'Division': 'Division',
    'Location': 'Location',
    'Costituenc': 'Constituency',
}, inplace=True)

print(data.columns)

# **PHASE 5: DIAGNOSTIC ANALYSIS**

## 5.1 Infrastructure Deficit

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
sns.boxplot(data=data, y='Pupil_Teacher_Ratio', ax=axes[0]).set_title('Teacher Ratio')
sns.boxplot(data=data, y='Pupil_Toilet_Ratio', ax=axes[1]).set_title('Toilet Ratio')
sns.boxplot(data=data, y='Pupil_Classroom_Ratio', ax=axes[2]).set_title('Classroom Ratio')
plt.suptitle("Distribution of Key Ratios (Raw Values)")
plt.show()

### Comparing ratios between school types

In [ ]:
# Compare ratios between school types
pd.pivot_table(data, 
               index='School_Status', 
               values=['Pupil_Teacher_Ratio', 'Pupil_Toilet_Ratio'],
               aggfunc=['mean', 'median', 'std'])

### Outlier Detection by Tukey's method

In [ ]:
# Tukey's method for outliers
def detect_outliers(col):
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    return data[(data[col] < (Q1 - 1.5*IQR)) | (data[col] > (Q3 + 1.5*IQR))]

print("Teacher ratio outliers:\n", detect_outliers('Pupil_Teacher_Ratio')['School_Name'].head())

## 5.2 Pupil-Teacher Ratio -continuos target

### 5.2.1 Spatial patterns

In [ ]:
# Geographic heatmap (using raw coordinates)
plt.figure(figsize=(10,8))
sns.scatterplot(data=data, x='Longitude', y='Latitude', 
                hue='Pupil_Teacher_Ratio', palette='viridis', 
                size='Total_Enrollment', sizes=(20,200))
plt.title("Teacher Ratio Geographic Distribution (Raw Coordinates)")
plt.show()

### 5.2.2 Statistical Tests

In [ ]:
from scipy.stats import ttest_ind

public = data[data['School_Status']=='PUBLIC']['Pupil_Teacher_Ratio']
private = data[data['School_Status']=='PRIVATE']['Pupil_Teacher_Ratio']
t_stat, p_val = ttest_ind(public, private)

print(f"Public vs Private t-test: t={t_stat:.1f}, p={p_val:.4f}")

## 5.3 Gender Toilet Equity

### 5.3.1 Raw Count Comparison

In [ ]:
data['Total_Toilets'] = data['Num_Boys_Toilets'] + data['Num_Girls_Toilets']
data['Girls_Toilet_Pct'] = data['Num_Girls_Toilets'] / data['Total_Toilets']

plt.figure(figsize=(12,6))
sns.histplot(data['Girls_Toilet_Pct'], bins=30, kde=True)
plt.axvline(0.5, color='red', linestyle='--', label='Perfect Equity')
plt.title("Proportion of Toilets Allocated to Girls")
plt.legend()
plt.show()

### 5.3.2 Cross-Tabulation

In [ ]:
pd.crosstab(data['Student_Gender_Type'], 
            data['School_Sponsor'], 
            values=data['Girls_Toilet_Pct'], 
            aggfunc='mean').style.background_gradient(cmap='RdYlGn')

## 5.4 School Performance Tier

### 5.4.1 Ratio Correlation

In [ ]:
sns.jointplot(data=data, x='Pupil_Teacher_Ratio', y='Pupil_Toilet_Ratio', 
              kind='hex', gridsize=20)
plt.suptitle("Teacher vs Toilet Ratio Correlation (Raw Values)")
plt.tight_layout()
plt.show()

### A dashboard layout to distribution by
    1. Pupil classroom ratio nationwide
    2. Provincial comparison
    3. Interactive plotly
    4. facility adequacy matrxi

In [ ]:
plt.figure(figsize=(12, 6))
ax = sns.histplot(data['Pupil_Classroom_Ratio'], bins=50, kde=True, color='#1f77b4')
plt.axvline(x=40, color='red', linestyle='--', linewidth=2, label='Recommended Max (40:1)')
plt.title('National Distribution of Pupil-Classroom Ratios', pad=20, fontsize=14)
plt.xlabel('Students per Classroom', labelpad=10)
plt.ylabel('Number of Schools', labelpad=10)
plt.legend()

# Annotate key points
plt.annotate(f'{len(overcrowded)} schools\n({len(overcrowded)/len(data)*100:.1f}%) exceed standard',
             xy=(60, ax.get_ylim()[1]*0.8), xytext=(70, ax.get_ylim()[1]*0.7),
             arrowprops=dict(arrowstyle='->'), bbox=dict(boxstyle='round,pad=0.5', fc='white'))

plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Median ratios by province
province_order = data.groupby('Province')['Pupil_Classroom_Ratio'].median().sort_values().index
sns.barplot(data=data, y='Province', x='Pupil_Classroom_Ratio', 
            order=province_order, ax=ax1, color='#1f77b4', estimator='median')
ax1.axvline(x=40, color='red', linestyle='--')
ax1.set_title('Median Classroom Ratios by Province', pad=15)
ax1.set_xlabel('Students per Classroom')
ax1.annotate('Western shows\nbest performance', 
             xy=(32, 6), xytext=(20, 4),
             arrowprops=dict(arrowstyle='->'))

# Plot 2: Percentage exceeding standards
province_exceed = data.groupby('Province')['Pupil_Classroom_Ratio'].apply(lambda x: (x>40).mean()*100).sort_values()
province_exceed.plot(kind='barh', ax=ax2, color='#ff7f0e')
ax2.set_title('% Schools Exceeding 40:1 Standard', pad=15)
ax2.set_xlabel('Percentage of Schools')
ax2.axvline(x=15, color='grey', linestyle=':')

plt.tight_layout()
plt.show()


import plotly.express as px
fig = px.box(data, x='Province', y='Pupil_Classroom_Ratio', 
             title='Interactive Provincial Distribution of Classroom Ratios',
             color='Province',
             hover_data=['School_Name'],
             points="all")

fig.add_hline(y=40, line_dash="dot", line_color="red",
              annotation_text="Recommended Maximum", 
              annotation_position="top left")

fig.update_layout(
    yaxis_title="Students per Classroom",
    xaxis_title="",
    showlegend=False,
    height=600
)

fig.show()

# Create adequacy flags
data['Classroom_Adequate'] = data['Pupil_Classroom_Ratio'] <= 40
data['Toilet_Adequate'] = (data['Pupil_Toilet_Ratio'] <= 25) & (data['Girls_Toilet_Pct'].between(40,60))

adequacy_matrix = data.groupby(['Classroom_Adequate', 'Toilet_Adequate']).size().unstack()

plt.figure(figsize=(8, 6))
sns.heatmap(adequacy_matrix, annot=True, fmt=',d', cmap='YlOrRd',
            cbar_kws={'label': 'Number of Schools'})
plt.title('Joint Adequacy of Classrooms and Toilets', pad=15)
plt.xlabel('Toilet Facilities Adequate')
plt.ylabel('Classroom Space Adequate')
plt.xticks([0.5,1.5], ['No', 'Yes'], rotation=0)
plt.yticks([0.5,1.5], ['No', 'Yes'], rotation=0)
plt.tight_layout()
plt.show()